<a href="https://colab.research.google.com/github/MuhamaadUsman/Alex_classifier-for-Cups-and-Mugs/blob/main/Lama2%20For%20Name%20entity%20recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf
!pip install accelerate
!pip install python-dotenv
!pip install sentence_transformers
!pip install llama-index
!pip install langchain
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import ServiceContext
import torch
!pip install -i https://test.pypi.org/simple/ bitsandbytes
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
import warnings
warnings.filterwarnings("ignore")

!huggingface-cli login


Looking in indexes: https://test.pypi.org/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permiss

In [ ]:

system_prompt = "You are a NER retrival assistant. Your goal to get NER data from the given data"

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")



In [ ]:
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.37.5
    Uninstalling bitsandbytes-0.37.5:
      Successfully uninstalled bitsandbytes-0.37.5


In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Required library version not found: libbitsandbytes_cuda122.so. Maybe you need to compile it from source?
CUDA SETUP: Defaulting to libbitsandbytes_cpu.so...

================================================ERROR=====================================
CUDA SETUP: CUDA detection failed! Possible reasons:
1. CUDA driver not installed
2. CUDA not installed
3. You have multiple conflicting CUDA libraries
4. Required library not pre-compiled for this bitsandbytes release!
CUDA SETUP: If you compiled from source, try again with `make CUDA_VERSION=DETECTED_CUDA_VERSION` for example, `make CUDA_VERSION=113`.
CUDA SETUP: The CUDA version for the compile might depend on your conda install. Inspect CUDA version via `c

RuntimeError: ignored

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
import os
import pandas as pd
from googleapiclient.discovery import build
from google.colab import auth

# Authenticate
auth.authenticate_user()

# Create Google Drive service
drive_service = build('drive', 'v3')

# Function to extract information using query
def extract_information(content):
    index = VectorStoreIndex.from_documents(content, service_context=service_context)
    query_engine = index.as_query_engine()
    response = query_engine.query(""" get comapny name and all its exceutive/president/director name,
     age and position held details from the document and send output as dict like this
     {'Company Name': 'Example Company','Executive Details': 'John Doe - CEO\nJane Smith - CFO'},
     executive info is usually in item 10. check item 10 if not found, find some where else if
      no excetive info found return No data found in Executive details""")

    return response

# Folder ID from the shared link
folder_id = ''  # Replace with your folder ID

# Create an empty DataFrame to store extracted data
df = pd.DataFrame(columns=['File Name', 'Company Name', 'Year', 'Quarter', 'Executive Details'])

page_token = None
while True:
    # List files in the folder with a page token
    results = drive_service.files().list(q=f"'{folder_id}' in parents and trashed=false",
                                         fields="nextPageToken, files(id, name)",
                                         pageSize=100,
                                         pageToken=page_token).execute()
    items = results.get('files', [])

    for item in items:
        file_id = item['id']
        file_name = item['name']

        # Access the file content
        request = drive_service.files().get_media(fileId=file_id)
        content = request.execute().decode('utf-8')

        # Replace 'YOUR_YEAR' and 'YOUR_QUARTER' with the actual year and quarter you want
        year = '2022'
        quarter = 'Q3'

        # Extract information using the query function
        extracted_info = extract_information(content)

        # Append extracted data to DataFrame
        df = df.append({
            'File Name': file_name,
            'Company Name': extracted_info.get('Company Name', None),
            'Year': year,
            'Quarter': quarter,
            'Executive Details': extracted_info.get('Executive Details', None),
        }, ignore_index=True)

    page_token = results.get('nextPageToken')
    if not page_token:
        break  # No more files

# Display the resulting DataFrame
print(df.head())

# Save the DataFrame to CSV
df.to_csv('/content/extracted_data/documents_info.csv', index=False)


In [ ]:
documents = SimpleDirectoryReader("/content/Data/").load_data()
index = VectorStoreIndex.from_documents(content, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query(""" get comapny name and all its exceutive/president/director name,
  age and position held details from the document and send output as dict like this
  {'Company Name': 'Example Company','Executive Details': 'John Doe - CEO\nJane Smith - CFO'},
  executive info is usually in item 10. check item 10 if not found, find some where else if
  no excetive info found return No data found in Executive details""")

In [ ]:
print(response)

Sure, here are the names, ages, and positions held by the individuals mentioned in the document:

• Vaughn R. Blake: 43, Managing Director
• Victor C. Chigas: 66, Executive Vice President and Director
• Mark J. Grant: 42, Executive Vice President and Director
• Lawrence R. Helfand: 57, Executive Vice President and Director
• Kenneth M. Karmin: 32, Director
• Kurt B. Karmin: 65, Chairman of the Board of Directors and Chief Executive Officer
• Scott H. Lang: 46, Executive Vice President and Director
• Gregory P. Quinlivan: 33, Secretary, General Counsel, and Executive Vice President
• Peter J. Schild: 46, Chief Financial Officer and Executive Vice President

Please let me know if you need any further assistance.


In [ ]:
# Install necessary libraries
!pip install google-auth google-auth-oauthlib google-auth-httplib2
import pandas as pd
import re
import os

from google.colab import auth
from googleapiclient.discovery import build

In [ ]:
import os
import pandas as pd
from google.colab import auth
from googleapiclient.discovery import build

# Authenticate
auth.authenticate_user()

# Create Google Drive service
drive_service = build('drive', 'v3')

# Folder ID from the shared link
folder_id = ''  # Replace with your folder ID

# Create an empty DataFrame to store extracted data
df = pd.DataFrame(columns=['File Name', 'Year', 'Quarter', 'Document Text'])

page_token = None
while True:
    # List files in the folder with a page token
    results = drive_service.files().list(q=f"'{folder_id}' in parents and trashed=false",
                                         fields="nextPageToken, files(id, name)",
                                         pageSize=100,
                                         pageToken=page_token).execute()
    items = results.get('files', [])

    for item in items:
        file_id = item['id']
        file_name = item['name']

        # Replace 'YOUR_YEAR' and 'YOUR_QUARTER' with the actual year and quarter you want
        year = '2022'
        quarter = 'Q1'

        # Access the file content
        request = drive_service.files().get_media(fileId=file_id)
        content = request.execute().decode('utf-8')

        # Append extracted data to DataFrame
        df = df.append({'File Name': file_name,
                        'Year': year,
                        'Quarter': quarter,
                        'Document Text': content}, ignore_index=True)

    page_token = results.get('nextPageToken')
    if not page_token:
        break  # No more files

# Create directory if it doesn't exist
directory = '/content/extracted_data'
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame to CSV in the created directory
df.to_csv('/content/extracted_data/documents_data.csv', index=False)

# Display the resulting DataFrame
print("Done")


<ipython-input-26-7387518ad4ab>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': file_name,
<ipython-input-26-7387518ad4ab>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': file_name,
<ipython-input-26-7387518ad4ab>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': file_name,
<ipython-input-26-7387518ad4ab>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': file_name,
<ipython-input-26-7387518ad4ab>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con